In [1]:
import carla
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Normal
import learn2learn as l2l
import random
import csv
import time


import torch.distributions as td

import torch.nn as nn
import torch.nn.functional as F

class NeuralNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(NeuralNetwork, self).__init__()

        self.l1 = nn.Linear(state_dim, 64)
        self.l2 = nn.Linear(64, 64)
        self.l3 = nn.Linear(64, action_dim)

    def forward(self, state):
        x = F.relu(self.l1(state))
        x = F.relu(self.l2(x))
        actions = self.l3(x)

        return actions


class CarlaEnvironment:
    def __init__(self, world, vehicle_blueprint, waypoints):
        self.world = world
        self.vehicle_blueprint = vehicle_blueprint
        self.waypoints = waypoints
        self.actorlist = []
        self.current_waypoint_list = []
        self.vehicle = self.world.spawn_actor(self.vehicle_blueprint, carla.Transform(carla.Location(x=-23.6,y=137.5,z=1),carla.Rotation(yaw=0)))
        
    def reset(self):
        try: self.vehicle.destroy()
        except: pass
        self.vehicle = self.world.spawn_actor(self.vehicle_blueprint, carla.Transform(carla.Location(x=-23.6,y=137.5,z=1),carla.Rotation(yaw=0)))
        self.current_waypoint = 0
        self.current_waypoint_list.append(self.current_waypoint)
        self.moving = False
        return self.get_state()

    def step(self, action):
        action = action.detach().numpy()
        self.current_waypoint_list.append(self.current_waypoint)
        #print(action)
        # Apply actions: Throttle, Steering, Brake
        if self.current_waypoint <10:
            action = [1.0, 0.0, 0.0]
        if self.current_waypoint ==10:
            self.moving = True            
        self.vehicle.apply_control(carla.VehicleControl(throttle = float(action[0]) , steer = float(action[1]) , brake = float(action[2])))
       
        #self.actorlist.append(self.vehicle)

        # Get next state
        next_state = self.get_state()

        # Calculate reward
        reward = self.calculate_reward(next_state)
       
        # Check if the episode is done
        done = self.is_done(next_state)

        return next_state, reward, done

#     def get_state(self):
#         curr_vel = self.vehicle.get_velocity()
#         curr_yaw = self.vehicle.get_transform().rotation.yaw
#         waypoint = self.waypoints[self.current_waypoint]
#         return np.array([curr_vel, curr_yaw - waypoint[1], curr_yaw - waypoint[2], waypoint[3]])
    
    def get_state(self):
        state = []
        curr_yaw = self.vehicle.get_transform().rotation.yaw
        
        
        for i in range(10):
            if self.current_waypoint + i < len(self.waypoints):
                waypoint = self.waypoints[self.current_waypoint + i]
                curr_vel = self.vehicle.get_velocity()
                curr_loc = self.vehicle.get_location()
                vx = curr_vel.x
                vy = curr_vel.y
                v = ((vx)**2+(vy)**2)**0.5
                if curr_loc.x==0 and curr_loc.y==0:
                    curr_loc = carla.Location(x=waypoint[4], y=waypoint[5])
                distance = curr_loc.distance(carla.Location(x=waypoint[4],y=waypoint[5]))
                
                state.extend([v, curr_yaw - waypoint[1], curr_yaw - waypoint[2], distance ])
            else:
                state.extend([0, 0, 0, 0])
               
        return np.array(state)

    def generate_waypoints(self):
        # Implement waypoint generation logic
        pass

    def calculate_reward(self, state):
        reward = -(np.linalg.norm(state[0] - self.waypoints[self.current_waypoint][0]) - (np.linalg.norm (state[1])) - (np.linalg.norm ( state[2])) - (np.linalg.norm (state[3])) + self.current_waypoint)
        return reward

    def is_done(self, state):
        curr_loc = self.vehicle.get_location()
        if curr_loc.x==0 and curr_loc.y==0:
            curr_loc = carla.Location(x=self.waypoints[self.current_waypoint][4],y=self.waypoints[self.current_waypoint][5])
 
        distance = curr_loc.distance(carla.Location(x=self.waypoints[self.current_waypoint][4],y=self.waypoints[self.current_waypoint][5]))

        if distance < 1:  # If the vehicle is close to the next waypoint
            self.current_waypoint += 1

        if self.current_waypoint >= len(self.waypoints):
            return True
    
        
        return False

    def set_weather(self, weather):
        self.world.set_weather(weather)
        
    def destroy(self):
        for actor in self.actorlist:
            actor.destroy()
        print("All Cleared")

def set_conditions(env, condition):
    
        precipitation = condition[0]
        precipitation_deposits = condition[1]
        tire_friction = condition[2]
        weather = carla.WeatherParameters( precipitation = precipitation,
                                            precipitation_deposits = precipitation_deposits)
        world.set_weather(weather)

        tire_condition = carla.WheelPhysicsControl(tire_friction = tire_friction)

        front_left_wheel  = carla.WheelPhysicsControl(tire_friction = tire_friction)
        front_right_wheel = carla.WheelPhysicsControl(tire_friction = tire_friction)
        rear_left_wheel   = carla.WheelPhysicsControl(tire_friction = tire_friction)
        rear_right_wheel  = carla.WheelPhysicsControl(tire_friction = tire_friction)

        wheels = [front_left_wheel, front_right_wheel, rear_left_wheel, rear_right_wheel]
        
        physics_control = env.vehicle.get_physics_control()
        physics_control.wheels = wheels
        env.vehicle.apply_physics_control(physics_control)

def train_ppo_maml(env, state_dim, action_dim, lr=1e-4, n_iters=1000000, inner_loop_steps=5):
    
    p_low = 0
    p_mid = 50
    p_high = 100

    pd_low = 50
    pd_high = 100

    tf_low = 0.5
    tf_mid = 1.5
    tf_high = 2.5
    
    conditions = [[p_low, pd_low, tf_high],
            [p_low, pd_low, tf_mid],
            [p_low, pd_high, tf_low],
            [p_mid, pd_low, tf_low],
            [p_mid, pd_high, tf_low],
            [p_mid, pd_low, tf_mid],
            [p_high, pd_low, tf_high],
            [p_high, pd_high, tf_low],
            [p_low, pd_high, tf_mid],
            [p_high, pd_low, tf_mid]]
    
    n_tasks = len(conditions)

    
    
    base_model = NeuralNetwork(state_dim, action_dim)
    maml = l2l.algorithms.MAML(base_model, lr=lr)
    optimizer = optim.Adam(maml.parameters(), lr=lr)
   
    iteration = 1
    for i in range(n_iters):
        condition = random.choice(conditions)
        set_conditions(env,condition)
    
        trajectories, state_tensors, action_tensors, rewards = sample_trajectories(env, maml)
        state_tensors_stacked = torch.stack(state_tensors)
        action_tensors_stacked = torch.stack(action_tensors)
        
        for _ in range(inner_loop_steps):
        
            learner = maml.clone()
            log_probs = compute_log_probs(learner, state_tensors_stacked, action_tensors_stacked)
            loss = policy_loss(log_probs, rewards)
            
            # error = nn.MSELoss()(learner(state_tensors_stacked), action_tensors_stacked)

            learner.adapt(loss)
            
            #error = nn.MSELoss()(learner(state_tensors_stacked), action_tensors_stacked)
            log_probs = compute_log_probs(learner, state_tensors_stacked, action_tensors_stacked)
            loss = policy_loss(log_probs, rewards)
            #error.backward(retain_graph=True)

        # Evaluating the model
        p_evaluate = random.uniform(p_low, p_high)
        pd_evaluate = random.uniform(pd_low, pd_high)
        tf_evaluate = random.uniform(tf_low, tf_high)
        evaluating_condition = [p_evaluate, pd_evaluate, tf_evaluate]
        set_conditions(env, evaluating_condition)
        
        trajectories, state_tensors, action_tensors, rewards = sample_trajectories(env, maml)
        state_tensors_stacked = torch.stack(state_tensors)
        action_tensors_stacked = torch.stack(action_tensors)

        log_probs = compute_log_probs(learner, state_tensors_stacked, action_tensors_stacked)
        loss = policy_loss(log_probs, rewards)
        loss.backward(retain_graph=True)

        # error = nn.MSELoss()(learner(state_tensors_stacked), action_tensors_stacked)
        # error.backward(retain_graph=True)
        
        
        optimizer.zero_grad()
        optimizer.step()
        
        iteration += 1
        print(f"Iteration: ", iteration)

        #if iteration % 1000 == 0:
        #    print("Iteration for {condition}: ", iteration)

def sample_weather_conditions(num_weather_conditions):
    return carla.WeatherParameters.HardRainNoon

def compute_log_probs(policy, states, actions):
    # states = torch.stack(states)
    # actions = torch.stack(actions)
    action_logits = policy(states)
    action_probs = F.softmax(action_logits,  dim=1)
    action_indices = actions.argmax(dim=1)

    log_probs = torch.log(action_probs[torch.arange(len(action_indices)), action_indices])
    
    return log_probs

    # std = log_std.exp()
    # normal_dist = td.Normal(mu, std)
    # log_probs = normal_dist.log_prob(actions).sum(dim=-1)
    return log_probs

def policy_loss(log_probs, rewards, gamma=0.99):
    discounted_rewards = []
    cumulative_reward = 0
    for reward in reversed(rewards):
        cumulative_reward = reward + gamma * cumulative_reward
        discounted_rewards.insert(0, cumulative_reward)
    discounted_rewards = torch.tensor(discounted_rewards)
    loss = -torch.mean(log_probs * discounted_rewards)
    return loss


def sample_trajectories(env, model, num_episodes=10, max_episode_length=1000):
#     model.to("cpu")
    trajectories = []
    traj_states = []
    traj_action = []
    state_tensors = []
    action_tensors = []
    rewards = []

    for _ in range(num_episodes):
        
        state = env.reset()              
        trajectory = []
        print("trajectories: ",len(trajectories))
        while True:
            
#             state_tensor = torch.tensor(state).to(torch.float32)
            state_tensor = torch.from_numpy(state.astype(np.float32))
            state_tensors.append(state_tensor)
            action_tensor = model(state_tensor)
#             action_distribution = Normal(action_mean, torch.tensor(0.1))
#             action = action_distribution.sample().detach().numpy().astype(float)
            
            action = action_tensor
#             action_tensor = torch.tensor(action, dtype=torch.float32)
            action_tensors.append(action)

            next_state, reward, done = env.step(action)
            trajectory.append((state, action, reward, next_state, done))
            rewards.append(reward)
            state = next_state
            waypoint_copy = env.current_waypoint
            # if not j%200 : print("Reward: ", reward, )

            if len(env.current_waypoint_list) >=51 and env.current_waypoint >10:
                if env.current_waypoint_list[len(env.current_waypoint_list)-1] == env.current_waypoint_list[len(env.current_waypoint_list) - 50] and env.moving:
                    done = True
            #state_tensors = torch.tensor(state_tensor, dtype=torch.float32)
            if done or (env.current_waypoint == len(env.waypoints) - 1):
                #env.vehicle.destroy()
                break  

        trajectories.append(trajectory)
    return trajectories, state_tensors, action_tensors, rewards

            
def read_waypoints(file_path):
    waypoints = []
    with open(file_path, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            v, yaw_c, yaw_n, r, x, y = map(float, row)
            waypoints.append((v, yaw_c, yaw_n, r, x, y))
            
    return waypoints                        


In [2]:
if __name__ == '__main__':

    client = carla.Client('localhost', 2000)
    client.set_timeout(2.0)

    world = client.get_world()
    blueprint_library = world.get_blueprint_library()
    vehicle_blueprint = blueprint_library.filter("wrangler_rubicon")[0]

    waypoints = read_waypoints("waypoint_with_xy.csv")
    env = CarlaEnvironment(world, vehicle_blueprint, waypoints)  # Implement CarlaEnvironment with the required state and action spaces

    state_dim = 40
    action_dim = 3

    train_ppo_maml(env, state_dim, action_dim)

        

trajectories:  0
trajectories:  1
trajectories:  2
trajectories:  3
trajectories:  4
trajectories:  5
trajectories:  6
trajectories:  7


: 

: 